#  Hyperparameter Tunning of Gradient Boosted Tree Model
Based on our initial assessment of model families, it appears as though the ensemble techinques of decisions trees tends to significantly outperform all other models. In this section I am going to dive deeper into two of the ensemble techniques. Random Forest and Gradient Boosted Trees. We are going to use this notebook to accomplish multiple tasks.

## Tasks
1) Hyperparameter test using Grid Search Cross Validation to try and get better performance out of these ensemble techniques. (It could be that the baseline hyperparameters are already optimized but we will confirm this)
2) Perform feature importance analysis using the decision tree (Mean-Decrease Impurity) to potentially improve our model
3) Perform Ablation analysis to show decrease in performance when removing specific features  
4) Conduct a failure analysis as to why certain companies perform poorly in our model (hypothesis are welcome)

## Import Libraries
Let's start as we usually do by importing the libraries we will be using in this section.

In [ ]:
# Data Manipulation libraries
import numpy as np
import pandas as pd

# Sci-Kit Learn Processing and Evaluating
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, r2_score

# Supervised Learning Models  
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor  
from sklearn.ensemble import GradientBoostingRegressor


# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

# Let's set our Random State and k_folds here as well
random_state = 42
k_fold = 10

## Import Data
Okay, now that we are working on our true model we are going to import training and test data so that we can see how we do.

In [2]:
root_path = '../../datasets/'
X_train_file = 'X_train_filled_KPIs_QoQ_PCA.csv'
y_train_file = 'y_train.csv'
X_test_file = 'X_test_filled_KPIs_QoQ_PCA.csv'
y_test_file = 'y_test.csv'
X_tr = pd.read_csv(root_path+X_train_file)
y_tr = pd.read_csv(root_path+y_train_file)
X_te = pd.read_csv(root_path+X_test_file)
y_te = pd.read_csv(root_path+y_test_file)
print(X_tr.shape, y_tr.shape,X_te.shape,y_te.shape)


(1910, 326) (1974, 19) (479, 326) (494, 19)


Once, again, we are dealing with these issues of abnormal numbers between the two datasets because we dropped data from the X_train and X_test file without addressing them in the y_files. So, let's go through and qualize both of these like we did in the previous notebooks.

In [3]:
in_train = set(X_tr['Ticker'])
in_test = set(X_te['Ticker'])
y_tr = y_tr[y_tr['Ticker'].isin(in_train)].copy()
y_te = y_te[y_te['Ticker'].isin(in_test)].copy()
print(f"length of in_train set: {len(in_train)}; length of in_test set: {len(in_test)}")
print(y_tr.shape,y_te.shape)

length of in_train set: 1910; length of in_test set: 479
(1910, 19) (479, 19)


In [4]:
y_tr = y_tr[['Ticker','Revenue_2025Q2','NetIncome_2025Q2']]
y_te = y_te[['Ticker','Revenue_2025Q2','NetIncome_2025Q2']]
print(y_tr.shape,y_te.shape)
y_tr.dropna(inplace=True)
y_te.dropna(inplace=True)
print(y_tr.shape,y_te.shape)
iny_tr = set(y_tr['Ticker'])
iny_te = set(y_te['Ticker'])
X_tr = X_tr[X_tr['Ticker'].isin(iny_tr)].copy()
X_te = X_te[X_te['Ticker'].isin(iny_te)].copy()
print(X_tr.shape, X_te.shape)
print(y_tr.shape, y_te.shape)

(1910, 3) (479, 3)
(1388, 3) (344, 3)
(1388, 326) (344, 326)
(1388, 3) (344, 3)


Okay, we have now equated the two. Let's make sure they are both in the right order as well.

In [5]:
print(X_tr['Ticker'].head(20))
print(y_tr['Ticker'].head(20))

1     HONE
3     RBRK
4      CSL
5      BMI
8     BLKB
9     PARR
11     CNH
12     XEL
14     NSA
15     AIZ
16     DKS
17     PEG
18      CI
19      FG
20    STRA
21      MP
23    ADCT
25     PFE
26    APPN
27     WSR
Name: Ticker, dtype: object
1     HONE
3     RBRK
5      CSL
6      BMI
11    BLKB
12    PARR
15     CNH
16     XEL
18     NSA
19     AIZ
21     DKS
22     PEG
23      CI
24      FG
25    STRA
26      MP
28    ADCT
30     PFE
31    APPN
32     WSR
Name: Ticker, dtype: object


## Hyperparameter Tuning
Alright, now that we have all of our data imported and ready to go, it's time to setup the GridSearchCV in order to tune our hyper parameters. This is going to be slightly dependent on which model we are using so let's set up a different one for each the Random Forest (RF) and Gradient Boosting Trees (GBT). Interestingly, because Tree's don't need scaling, I am going to avoid much of the transformations that we did in the earlier notebook to see how we do on raw values. 

Random Forest Hyperparameters:
- n_estimators  
- max_depth  
- min_samples_leaf  
- min_samples_split  
- max_features  
- bootstrap  
- max_samples

Gradient Boosting Hyperparameters:  
- learning_rate  
- n_estimators  
- max_depth  
- min_samples_leaf  
- min_samples_split  
- max_features  
- subsample  
- loss  
- validation_fraction  
- n_iter_no_change  
- tol

## Hyperparameter Tuning Pipeline
Okay, now that we have defined all of that we can set up our pipeline to run through the hyperparameter tuning for us and see what is best. First, lets drop the name and ticker column as they won't provide anything useful in our datasets.

In [6]:
# Drop the ticker and name columns so we don't explode the feature space unneccesarily
X_tr.drop(columns=['Ticker','Name'],axis=1,inplace=True)
X_te.drop(columns=['Ticker','Name'],axis=1,inplace=True)

In [ ]:
# identify categorical columns using dtypes
cat_cols = [c for c in X_tr.columns if str(X_tr[c].dtype) in ("object", "category")]
pre = ColumnTransformer(
    [("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
    remainder="passthrough",
)

# base pipeline for gridsearch
pipe = Pipeline([("pre", pre),
                 ("model", RandomForestRegressor(random_state=random_state, n_jobs=-1))])

# Have a couple of different scorers for analyzing
def rmse(y_true, y_pred): return mean_squared_error(y_true, y_pred, squared=False)
rmse_s = make_scorer(rmse, greater_is_better=False)
mae_s  = make_scorer(mean_absolute_error, greater_is_better=False)

scoring = {"rmse": rmse_s, "mae": mae_s, "r2": "r2"}
cv = KFold(n_splits=k_fold, shuffle=True, random_state=random_state)

# Create our grids for the gridsearch (including our two models)
gbt = GradientBoostingRegressor(random_state=random_state)

gbt_grid = {
    "model": [gbt],
    "model__learning_rate": [0.07, 0.1, 0.15],
    "model__n_estimators": [100, 200, 500],
    "model__max_depth": [2, 3, 4],
    "model__min_samples_leaf": [1, 5, 20],
    "model__min_samples_split": [2, 10, 50],
    "model__max_features": ["sqrt", "log2", 0.3, 0.5, 0.7],
    "model__subsample": [0.6, 0.8, 1.0],
    "model__loss": ["squared_error", "absolute_error"],
}

# Run the gridsearch and we are going to choose best RMSE
grid = GridSearchCV(
    estimator=pipe,
    param_grid=[gbt_grid],
    scoring=scoring,
    refit="r2",          
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=False,
)

y_series = np.abs(y_tr["Revenue_2025Q2"])
grid.fit(X_tr, y_series)

print("Best params:", grid.best_params_)
print(f"Best R2: {grid.best_score_:.3f}")

Fitting 10 folds for each of 7290 candidates, totalling 72900 fits


c:\Users\tmacp\anaconda3\envs\milestoneII\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


Best params: {'model': GradientBoostingRegressor(random_state=0), 'model__learning_rate': 0.15, 'model__loss': 'squared_error', 'model__max_depth': 3, 'model__max_features': 0.5, 'model__min_samples_leaf': 1, 'model__min_samples_split': 50, 'model__n_estimators': 200, 'model__subsample': 1.0}
Best R2: 0.983


Alright, now that we have pulled out the Test set parameters: 

Best params: {'model': GradientBoostingRegressor(random_state=0), 'model__learning_rate': 0.15, 'model__loss': 'squared_error', 'model__max_depth': 3, 'model__max_features': 0.5, 'model__min_samples_leaf': 1, 'model__min_samples_split': 50, 'model__n_estimators': 200, 'model__subsample': 1.0}

We can start to get all of the data from it.

In [20]:
# --- data columns ---
cat_cols = ['Sector', 'Exchange', 'Market Cap']

# y_tr_col/y_te_col are your revenue targets, e.g. 'Revenue_2025Q2'
y_tr_series = y_tr['Revenue_2025Q2']
y_te_series = y_te['Revenue_2025Q2']

# --- preproc ---
pre = ColumnTransformer(
    [("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
    remainder="passthrough",
)

# --- tuned model ---
gbr = GradientBoostingRegressor(
    random_state=random_state,
    learning_rate=0.15,
    loss="squared_error",
    max_depth=3,
    max_features=0.5,
    min_samples_leaf=1,
    min_samples_split=50,
    n_estimators=200,
    subsample=1.0,
)

pipe = Pipeline([("pre", pre), ("model", gbr)])

# 
cv = KFold(n_splits=k_fold, shuffle=True, random_state=random_state)



scores = cross_validate(
    pipe, X_tr, y_tr_series, cv=cv, n_jobs=-1, return_train_score=False,
    scoring={"rmse": "neg_root_mean_squared_error",
             "mae":  "neg_mean_absolute_error",
             "r2":   "r2"},
    error_score="raise"   # surface any hidden exceptions
)
rmse_cv = -scores["test_rmse"]
mae_cv  = -scores["test_mae"]
r2_cv   =  scores["test_r2"]

cv_summary = pd.DataFrame({
    "metric": ["RMSE","MAE","R2"],
    "mean":   [np.round(rmse_cv.mean(),2), np.round(mae_cv.mean(),2), r2_cv.mean()],
    "std":    [rmse_cv.std(ddof=1), mae_cv.std(ddof=1), r2_cv.std(ddof=1)],
})

# --- test-set metrics (single numbers) ---
pipe.fit(X_tr, y_tr_series)
y_pred = pipe.predict(X_te)

rmse_test = np.sqrt(mean_squared_error(y_te_series, y_pred))  # RMSE
mae_test  = mean_absolute_error(y_te_series, y_pred)
r2_test   = r2_score(y_te_series, y_pred)

test_summary = pd.Series({"RMSE": rmse_test, "MAE": mae_test, "R2": r2_test})


print("\nTest set:", test_summary.to_dict())
cv_summary




Test set: {'RMSE': 1624030050.132752, 'MAE': 419818194.50718665, 'R2': 0.9766423863518396}


,metric,mean,std
0,RMSE,2.011170e+09,9.628414e+08
1,MAE,3.774524e+08,1.028788e+08
2,R2,9.586107e-01,3.169618e-02


Alright now, let's calculate success rates for the test set.

In [21]:
def success_rates(y_true, y_pred, thresholds=(0.10, 0.20, 0.40)):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    mask = y_true != 0  # avoid div-by-zero; report how many dropped
    rel_err = np.empty_like(y_true, dtype=float)
    rel_err[mask] = np.abs(y_pred[mask] - y_true[mask]) / np.abs(y_true[mask])
    rel_err[~mask] = np.nan

    out = {"n_used": int(mask.sum()), "n_zero_actuals": int((~mask).sum())}
    for t in thresholds:
        out[f"≤{int(t*100)}%"] = np.nanmean(rel_err <= t)
    return pd.Series(out)

# example
rates = success_rates(y_te_series, y_pred)  # after you predict
print(rates)

n_used            344.000000
n_zero_actuals      0.000000
≤10%                0.491279
≤20%                0.750000
≤40%                0.880814
dtype: float64


alright, this is pretty darn good. Let's plot everything to see what it looks like.

In [36]:
# Data
dfp = pd.DataFrame({
    "actual": np.asarray(y_te_series, dtype=float),
    "pred":   np.asarray(y_pred, dtype=float)
})
dfp["resid"] = dfp["actual"] - dfp["pred"]  # residual = y - ŷ

# R² for annotation
r2 = r2_score(dfp["actual"], dfp["pred"])

# ---------- Residuals vs Actual ----------
res_scatter = alt.Chart(dfp).mark_circle(opacity=0.4,color='Magenta',size=60).encode(
    x=alt.X("actual:Q", title="Actual", axis=alt.Axis(grid=False)),
    y=alt.Y("resid:Q",  title="Residual (y - ŷ)", axis=alt.Axis(grid=False))
).properties(width=500, height=300, title="Residuals vs Actual")

zero_line = alt.Chart(pd.DataFrame({"y":[0]})).mark_rule().encode(y="y:Q")

residuals_chart = res_scatter + zero_line

# ---------- Predicted vs Actual with 45° line + R² ----------
xy_min = float(min(dfp["actual"].min(), dfp["pred"].min()))
xy_max = float(max(dfp["actual"].max(), dfp["pred"].max()))
diag_df = pd.DataFrame({"x":[xy_min, xy_max], "y":[xy_min, xy_max]})

scatter_xy = alt.Chart(dfp).mark_circle(opacity=0.4,color='Magenta',size=60).encode(
    x=alt.X("actual:Q", title="Actual", axis=alt.Axis(grid=False)),
    y=alt.Y("pred:Q",   title="Predicted (ŷ)", axis=alt.Axis(grid=False))
).properties(width=500, height=300, title="Predicted vs Actual")

diag = alt.Chart(diag_df).mark_line(color='black').encode(x="x:Q", y="y:Q")

# R² text placed near top-left
annot_df = pd.DataFrame({"x":[xy_min], "y":[xy_max], "label":[f"R² = {r2:.3f}"]})
r2_text = alt.Chart(annot_df).mark_text(align="left", baseline="top", dx=6, dy=6).encode(
    x="x:Q", y="y:Q", text="label:N"
)

pred_vs_actual_chart = scatter_xy + diag + r2_text

(pred_vs_actual_chart | residuals_chart).configure_view(stroke=None)

alt.HConcatChart(...)